In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder

In [3]:
train_data = pd.read_csv('C:/Users/MSI/Downloads/train.csv')
test_data = pd.read_csv('C:/Users/MSI/Downloads/test.csv')

In [4]:
X_train = train_data.drop(columns=['PassengerId', 'Survived','Name', 'Ticket', 'Cabin'], axis=1)
X_train['Embarked'].fillna('S', inplace=True)
le = LabelEncoder()
X_train['Embarked'] = le.fit_transform(X_train['Embarked']) 
X_train['Sex'] = le.fit_transform(X_train['Sex'])
X_train                          

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,2
887,1,0,19.0,0,0,30.0000,2
888,3,0,NaN,1,2,23.4500,2
889,1,1,26.0,0,0,30.0000,0


In [5]:
median_age = X_train['Age'].median()
X_train['Age'] = X_train['Age'].apply(lambda x: median_age if np.isnan(x) else x)
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,2
887,1,0,19.0,0,0,30.0000,2
888,3,0,28.0,1,2,23.4500,2
889,1,1,26.0,0,0,30.0000,0


In [6]:
y_train = train_data.Survived

Теперь предобработаем test:

In [7]:
X_test = test_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X_test['Embarked'] = le.fit_transform(test_data['Embarked'])
X_test['Sex'] = le.fit_transform(X_test['Sex'])
X_test['Age'] = X_test['Age'].apply(lambda x: median_age if np.isnan(x) else x)
median_fare = X_test['Fare'].median()
X_test['Fare'] = X_test['Fare'].apply(lambda x: median_age if np.isnan(x) else x)

Теперь построим ОГРОМНЫЙ СЛУЧАЙНЫЙ ЛЕС

In [8]:
rf = RandomForestClassifier()
parameters = {'criterion': ['gini', 'entropy'],
              'n_estimators': range(10, 500, 10),
             'max_depth': range(1, 500),
             'min_samples_split': range(2, 200),
             'min_samples_leaf': range(1, 200),
             }

In [9]:
rf_grid_clf = RandomizedSearchCV(rf, parameters, cv=5, n_jobs=-1)
rf_grid_clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [10]:
best_estimator = rf_grid_clf.best_estimator_
rf_grid_clf.best_params_

{'n_estimators': 350,
 'min_samples_split': 108,
 'min_samples_leaf': 2,
 'max_depth': 160,
 'criterion': 'entropy'}

In [11]:
predictions = best_estimator.predict(X_test)
best_estimator.score(X_train, y_train)

0.8282828282828283

In [12]:
from xgboost import XGBClassifier

In [14]:
xgb_model = XGBClassifier()
params = {
            'eta': np.arange(0.1, 0.26, 0.05),
            'min_child_weight': np.arange(1, 5, 0.5).tolist(),
            'gamma': [5],
            'subsample': np.arange(0.5, 1.0, 0.11).tolist(),
            'colsample_bytree': np.arange(0.5, 1.0, 0.11).tolist()
        }

In [15]:
grid = GridSearchCV(xgb_model, 
                    param_grid = params,  
                    n_jobs = -1, 
                    cv = 5)

In [17]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_l...
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 0.61, 0.72, 0.83, 0.94],
                         'eta': array([0.1 , 0.15, 0.2 , 0.25]), 'gamma': [5],
                   

In [22]:
predictions = grid.predict(X_test)

In [25]:
predictions = pd.DataFrame(predictions, columns=['Survived'])
result = pd.DataFrame(test_data['PassengerId'])
result = pd.concat([result, predictions], axis=1)
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [26]:
result.to_csv(path_or_buf='C:/Users/MSI/Desktop/Data Science/submission.csv', sep=',', index=False)

Точность 0.79425 и топ-18% на kaggle. XGBoost могёт, осталось корректно настроить параметры